# Products and the functor of chains: the EZ-AW bilax structure

### Abstract

We construct functions modeling the Alexander-Whitney map AW, the Eilenberg-Zilber map EZ, and chain homotopy from EZAW to the identity. This constructions use the SimplicialBioperator class.

## Simplicial sets and  the SimplicialOperator class 

A **[simplicial set](https://en.wikipedia.org/wiki/Simplicial_set)** $X$ is a collection of set $\{X_n\}_{n \geq 0}$ together with **degeneracy maps** and **face maps**
\begin{equation*}
s_i : X_n \to X_{n+1} \qquad d_i : X_n \to X_{n-1}
\end{equation*}
for $i = 0, \dots, n$ satisfying the **simplicial identities**:

\begin{align*}
d_i d_j &= d_{j-1} d_i  &\text{ if } i &< j.  \\
d_i s_j &= s_{j-1}d_i    &\text{ if } i &< j. \\
d_i s_j &= \text{id}  &\text{ if } i &= j \text{ or } i = j. \\
d_i s_j &= s_j d_{i-1}   &\text{ if } i &> j. \\
s_i s_j &= s_{j+1} s_i   &\text{ if } i &≤ j.
\end{align*}

We call an element $\sigma \in X_n$ a **simplex of dimension $n$** or **$n$-simplex**. We say that $\sigma$ is a **face** of a simplex $\tau$ if it is the image of $\tau$ via a composition of face maps. We say that $\sigma$ is **degenerate** if it is the image via a degeneracy map of any simplex. 

A **simplicial map** $F : X \to Y$ is a collection of functions $F_n : X_n \to Y_n$ commuting with degeneracy and face maps. 

The set of **simplicial operators** contains the equivalence classes of formal concatenations of symbols $s_i$ and $d_j$ modulo the simplicial identities. A simplicial operator can be uniquely written in the **canonical form**:

\begin{equation*}
s_{u_1} \cdots s_{u_p} d_{v_1} \cdots d_{v_q}
\end{equation*}
with $u_1 > \cdots > u_p$ and $v_1 < \cdots < v_q$.

A simplicial set is said to be **computable** if for any $n$-tuple of $0$-dimensional simplices there exists at most one simplex having it as its ordered collection of $0$-dimensional faces. Examples include: ordered simplicial complexes, directed simplicial complexes, and the nerve of categories.

If $[v_0, \dots, v_n]$ represents an $n$-simplex in a computable simplicial set. The action of any simplicial operator on it is induced from

\begin{equation}
d_i [v_0, \dots, v_n] = [v_0, \dots, \widehat{v}_i, \dots, v_n]
\end{equation}

\begin{equation}
s_i [v_0, \dots, v_n] = [v_0, \dots, v_i, v_i, \dots, v_n].
\end{equation}

In [1]:
class SimplicialOperator:
    '''
    Models a simplicial operator of the form: 

    s ... s d ... d
    
    represented in the canonical form
    
    s > ... > s d < ... < d

    Parameters
    ----------
    deg_maps  : tuple or list
                An ordered collection of integers representing the degeneracy maps of the operator
    
    face_maps : tuple or list
                An ordered collection of integers representing the face maps of the operator

    '''

    def __init__(self, deg_maps = [], face_maps = []):   
    
        self.deg_maps  = SimplicialOperator.deg_maps_sort(deg_maps) 
        self.face_maps = SimplicialOperator.face_maps_sort(face_maps)
    
    def __repr__(self):
    
        s, d = repr(self.deg_maps), repr(self.face_maps)
    
        return f'SimplicialOperator(deg_maps={s}, face_maps={d})'
  
    def __str__(self):
    
        if not self.deg_maps and not self.face_maps:
        
            return 'id'
    
        if not self.deg_maps:
            s = ''
        else:       
            s = f's_{"s_".join(str(i) for i in self.deg_maps)}'

        if not self.face_maps:
            d = ''
        else:
            d = f'd_{"d_".join(str(i) for i in self.face_maps)}'

        return(s+d)

    def __call__(self, simplex):
        '''applies the operator to a simplex represented by a list or a tuple'''
    
        simplex = list(simplex)
    
        if self.face_maps and len(simplex)-1 < self.face_maps[-1]:
            raise ValueError('simplex not in the domain of the operator')
    
        # face maps
        simplex = ([v for i, v in enumerate(simplex) 
                    if i not in self.face_maps])

        # degeneracy maps
        for i in sorted(self.deg_maps):
            try:
                simplex.insert(i+1, simplex[i]) 
            except IndexError:
                raise ValueError('simplex not in the domain of the operator')

        return simplex
    
    @property
    def degree(self):
        '''returns the degree of the operator as an int'''
    
        return len(self.deg_maps) - len(self.face_maps)
  
    @property
    def is_degenerate(self):
        '''returns True if the operator is degenerate and False otherwise'''
    
        return bool(self.deg_maps)
  
    def compose(self, other):
        '''returns the operator self other'''
    
        s_left, d_left  = list(self.deg_maps), list(self.face_maps)
        s_right, d_right = list(other.deg_maps), list(other.face_maps)

        # getting s_right passed d_left
        d = d_left
        s = []
        for j in s_right:
            if j+1 in d or j in d:
                p = max(filter(lambda k: k == j or k == j+1, d))
                d = d[:d.index(p)] + [k-1 for k in d[d.index(p)+1:]]

            else:
                lt = list(filter(lambda k: k < j, d))
                gt = list(filter(lambda k: k > j+1, d))
                d = lt + [k-1 for k in gt]
                s.append(j - len(lt))

        # One can maybe optimize in the next step using that 
        # s_left, s, d, and d_right are totally ordered

        new_deg_maps  = SimplicialOperator.deg_maps_sort(s_left + s) 
        new_face_maps = SimplicialOperator.face_maps_sort(d + d_right)

        return SimplicialOperator(new_deg_maps, new_face_maps)
  
    @staticmethod  
    def deg_maps_sort(deg_maps):
        '''puts the degeneracy maps in canonical order s > ... > s using 
        the simplicial identity s_i s_j = s_{j+1} s_i if i <= j'''

        deg_maps = list(deg_maps)
        for index in reversed(range(len(deg_maps)-1)):

            currentvalue = deg_maps[index]
            position = index

            while (position<len(deg_maps)-1 and
                   currentvalue <= deg_maps[position+1]):
                
                deg_maps[position] = deg_maps[position+1]+1
                position = position+1

            deg_maps[position] = currentvalue

        return tuple(deg_maps)
  
    @staticmethod
    def face_maps_sort(face_maps):
        '''puts the face maps in canonical order d < ... < d using the 
        simplicial identity d_i d_j = d_j d_{i+1} if i >= j'''

        face_maps = list(face_maps)
        for index in range(1, len(face_maps)):

            currentvalue = face_maps[index]
            position = index

            while (position > 0 and
                   face_maps[position-1] >= currentvalue):
                
                face_maps[position] = face_maps[position-1]+1
                position = position-1

            face_maps[position] = currentvalue

        return tuple(face_maps)

### Example

We will model the operator 
$$op = s_0s_1d_1d_0$$ 
whose action on the simplex $[0,1,2,3]$ is $[1, 1, 3, 3]$.

Its canonical representative is 
$$s_2s_0d_0d_2$$
and we have $op\ op = op$.

In [2]:
op = SimplicialOperator([0,1],[1,0])

print(f'The canonical representative {op} of our operator is stored.\n')

print(f'Its action on [0,1,2,3] is {op(range(4))}\n')

print(f'We verify this operator is idempotent by composing it with itself and obtaining {op.compose(op)}.')

The canonical representative s_2s_0d_0d_2 of our operator is stored.

Its action on [0,1,2,3] is [1, 1, 3, 3]

We verify this operator is idempotent by composing it with itself and obtaining s_2s_0d_0d_2.


## Chains with $\mathbb F_2$-coefficients, tensor products and the SimplicialBioperator class

Let us consider the **normalized chains with $\mathbb F_2$-coefficients** on a simplicial set 
\begin{equation*}
N_n(X) = \frac{\mathbb F_2 \{ X_n \}}{\mathbb F_2 \{ s(X_{n-1}) \}} \ \qquad
\partial_n = \sum_{i=0}^{n} d_{i}
\end{equation*}
where $s(X_{n-1}) = \bigcup_{i=0}^{n-1} s_i(X_{n-1})$. 

A **linear simplicial operators** is a formal linear combination of simplicial operators acting linearly on normalized chains.   

The **tensor product** of two $\mathbb F_2$-chain complexes is $C$ and $C'$ is defined by

\begin{equation}
(C \otimes C')_n = \bigoplus_{i+j=n} C_i \otimes C'_j\ \qquad
\partial = \partial \otimes \mathrm{id} + \mathrm{id} \otimes \partial
\end{equation}

A **simplicial bioperator** is the tensor product of two linear simplicial operators acting on $N_*(X) \otimes N_*(Y)$ bilinearly. 

We adapt the SimplicialOperator class to model simplicial bioperators of the form

\begin{equation*}
s_{u_1} \cdots s_{u_p} d_{v_1} \cdots d_{v_q}
\otimes
s_{r_1} \cdots s_{r_l} d_{s_1} \cdots d_{s_t.}
\end{equation*}	

In [61]:
class SimplicialBioperator:
    '''
    Models a simplicial bioperator of form: 

    s ... s d ... d (x) s ... s d ... d
    
    represented in the canonical form
    
    s > ... > s d < ... < d (x) s > ... > s d < ... < d

    Parameters
    ----------
    deg_maps1  : tuple or list
                 An ordered collection of integers representing the degeneracy maps of the first operator
    
    face_maps1 : tuple or list
                 An ordered collection of integers representing the face maps of the first operator
    
    deg_maps2  : tuple or list
                 An ordered collection of integers representing the degeneracy maps of the second operator
    
    face_maps2 : tuple or list
                 An ordered collection of integers representing the face maps of the second operator

    '''
    
    def __init__(self, deg_maps1=[], face_maps1=[], deg_maps2=[], face_maps2=[]):
    
        self.op1 = SimplicialOperator(deg_maps1, face_maps1)
        self.op2 = SimplicialOperator(deg_maps2, face_maps2)
    
    def __repr__(self):
        
        s1, d1 = repr(self.op1.deg_maps), repr(self.op1.face_maps)
        s2, d2 = repr(self.op2.deg_maps), repr(self.op2.face_maps)
        
        return (f'Bioperator(deg_maps1={s1}, face_maps1={d1}, deg_maps2={s2}, face_maps2={d2}')
    
    def __str__(self):

        return(str(self.op1) + ' x ' + str(self.op2))
    
    def __call__(self, spx1, spx2):
        
        return self.op1(spx1), self.op2(spx2)
    
    @property
    def bidegree(self):
        '''returns the degree of the operator as an int'''
    
        return self.op1.degree, self.op2.degree
    
    @property
    def is_degenerate(self):
        '''returns True if the operator is degenerate and False if not'''
        
        s1 = set(self.op1.deg_maps)
        s2 = set(self.op2.deg_maps)
        
        return bool(s1.intersection(s2))    
    
    def compose(self, other):
        '''returns the operator self other'''
        
        op1 = self.op1.compose(other.op1)
        op2 = self.op2.compose(other.op2)
        
        return ( SimplicialBioperator(op1.deg_maps, op1.face_maps, 
                                      op2.deg_maps, op2.face_maps) )

### Example

We will model the operator 
$$biop = s_0s_1d_1d_0 \otimes s_1 d_0 d_1$$ 
whose action on the simplex $[0,1,2,3]$ is $[1, 1, 3, 3]$.

Its canonical representative is 
$$s_2s_0d_0d_2 \otimes s_1d_0d_1$$
and we have $op\ op = op$.

In [62]:
biop = SimplicialBioperator([0,1],[1,0],[1],[0,1])

print(f'The canonical representative {biop} of our operator is stored.\n')

action = biop(range(4),range(5))
print(f'Its action on [0,1,2,3] x [0,1,2,3,4] is {action[0]} x {action[1]}\n')

print(f'The composition with itself is {biop.compose(biop)}.')

The canonical representative s_2s_0d_0d_2 x s_1d_0d_1 of our operator is stored.

Its action on [0,1,2,3] x [0,1,2,3,4] is [1, 1, 3, 3] x [2, 3, 3, 4]

The composition with itself is s_2s_0d_0d_2 x s_1d_0d_1d_2.


## Products and the EZ and AW maps

The product of two simplicial sets $X$ and $Y$ is defined by
\begin{equation}
(X \times Y)_n = X_n \times Y_n,
\end{equation}

\begin{equation}
d_i(x \times y) = d_i x \times d_i y,
\end{equation}

\begin{equation}
s_i(x \times y) = s_i x \times s_i y.
\end{equation}

We notice that $N_*(X \times Y) \not= N_*(X) \otimes N_*(Y)$. 

An **ordered partition** of a set $S$ is a tuple of disjoint subsets of $S$ whose union is $S$.

### The **Alexander-Whitney map**
\begin{equation*}
    AW: N_*(X \times Y) \to N_*(X) \otimes N_*(Y)
\end{equation*}
is defined for $x \times y \in N_*(X \times Y)_n$ by
\begin{equation*}
    AW(x \times y) = \sum_{i=0}^n d_{i+1} \cdots d_n\, x \otimes d_0 \cdots d_{i-1}\, y.
\end{equation*}




In [63]:
def aw(n, q=None):
    '''if an integer n is passed it returns the linear combination of bioperators defining the 
    restriction of AW to degree n. If two integers (p, q) are passed it provides the single 
    bioperad defining the projection to bidegree (p, q) of the AW map'''
    
    if not q is None:
        p = n
        return SimplicialBioperator(face_maps1=range(p+1,p+q+1), 
                                    face_maps2=range(0,p))
    if q is None:
        linear_comb = set()
        for i in range(n+1):
            j = n-i
            biop = SimplicialBioperator(face_maps1=range(i+1,i+j+1), 
                                        face_maps2=range(0,i))
            linear_comb ^= {biop}
    
        return linear_comb

### Example
For the map
\begin{equation*}
AW : N_*(\Delta^1 \times \Delta^2) \to N_*(\Delta^1) \otimes N_*(\Delta^2)
\end{equation*}
we verify that
\begin{equation*}
AW([0,0,0,1] \times [0,1,2,2]) = 
[0] \otimes [0,1,2,2] + [0,0] \otimes [1,2,2] + 
[0,0,0] \otimes [2,2] +[0,0,0,1] \otimes [2].
\end{equation*}

In [64]:
[biop([0,0,0,1], [0,1,2,2]) for biop in aw(3)]

[([0], [0, 1, 2, 2]),
 ([0, 0], [1, 2, 2]),
 ([0, 0, 0], [2, 2]),
 ([0, 0, 0, 1], [2])]

### The **Eilenberg-Zilber map**

\begin{equation*}
    EZ: N_*(X) \otimes N_*(Y) \to N_*(X \times Y)
\end{equation*}
is defined for $x \otimes y \in N_p(X) \otimes N_q(Y)$ by
\begin{equation*}
    EZ(x \otimes y) = 
    \sum s_{w_q} \cdots s_{w_1}\, x \times s_{v_p} \cdots s_{v_1}\, y
\end{equation*}
where the sum is over all ordered partitions 
\begin{equation*}
\big( \{v_1 < \cdots < v_p\},\ \{w_1 < \cdots < w_q\} \big)
\end{equation*}	
of $\{0,\dots,p+q-1\}$.

In [65]:
def ez(n, q=None):
    '''if a single integer n is passed, it returns the set of bioperators defining the 
    restriction of EZ to degree n. If two integers (p,q) are passed, it returns the  
    bioperators defining EZ in bidegree (p,q)'''
    
    # operators in bidegrees (0,n), (1,n-1), ... , (n,0) 
    if not q:
        if n == 0:
            return {SimplicialBioperator()}

        if n > 0:
            answer = set()
            for biop in ez(n-1):
                i,j = biop.bidegree
                answer ^= {SimplicialBioperator(
                                deg_maps1 = biop.op1.deg_maps,
                                deg_maps2 = [i+j] + list(biop.op2.deg_maps))}

                answer ^= {SimplicialBioperator(
                                deg_maps1 = [i+j] + list(biop.op1.deg_maps),
                                deg_maps2 = biop.op2.deg_maps)}
            return answer
        
    # operators in bidegree (p,q)
    if q:
        p = n
        if (p,q) == (0,0):
            return [SimplicialBioperator()]

        answer = set()
        if p > 0:
            west = ez(p-1, q)
            for biop in west:
                answer ^= {SimplicialBioperator(
                                deg_maps1 = biop.op1.deg_maps,
                                deg_maps2 = [p+q-1] + list(biop.op2.deg_maps))}
        if q > 0:
            south = ez(p, q-1)
            for biop in south:
                answer ^= {SimplicialBioperator(
                                deg_maps1 = [p+q-1] + list(biop.op1.deg_maps),
                                deg_maps2 = biop.op2.deg_maps)}
        return answer

### Example
For the map
\begin{equation*}
EZ : N_*(\Delta^1) \otimes N_*(\Delta^2) \to N_*(\Delta^1 \times \Delta^2)
\end{equation*}

we verify that

\begin{equation*}
EZ([0,1] \otimes [0,1,2]) = [0,0,0,1] \times [0,1,2,2] + [0,0,1,1] \times [0,1,1,2] + [0,1,1,1] \times [0,0,1,2].
\end{equation*}

In [66]:
[biop([0,1], [0,1,2]) for biop in ez(1,2)]

[([0, 0, 0, 0, 1], [0, 1, 2]),
 ([0, 0, 0, 1], [0, 1, 2, 2]),
 ([0, 1, 1, 1], [0, 0, 1, 2]),
 ([0, 0, 1, 1], [0, 1, 1, 2])]

### The chain homotopy $EZAW \sim id$

The composition $EZAW$ is chain homotopic to the identity via the map

\begin{equation*}
SHI_n : N_n(X \times Y) \to N_{n+1}(X \times Y)
\end{equation*}
recursively defined by 
\begin{equation*}
SHI_n = 
\begin{cases} 
0 & n = 0 \\
SHI'_{n-1} + EZAW ' (s_0 \otimes s_0) & n > 0
\end{cases}
\end{equation*}
where 
\begin{equation*}
\big( s_{v_{l}} \cdots s_{v_1} d_{i_1} \cdots d_{i_p} \big) ' = s_{v_{l}+1} \cdots s_{v_1+1} d_{i_1+1} \cdots d_{i_p+1}.
\end{equation*}

In [36]:
from itertools import product

def ezaw(n):
    '''returns the bioperator defining the map EZAW restricted to degree n'''
    comps = set()
    for p in range(n+1):
        q = n-p
        comps ^= {biop2.compose(biop1) for
                  biop2, biop1 in product(ez(p,q), [aw(p,q)])}
    return comps

def prime(biop):
    '''adds 1 to all the elements in the simplicial bioperator'''
    s1 = [v+1 for v in biop.op1.deg_maps]
    d1 = [w+1 for w in biop.op1.face_maps]
    s2 = [v+1 for v in biop.op2.deg_maps]
    d2 = [w+1 for w in biop.op2.face_maps]
    
    return SimplicialBioperator(deg_maps1 = s1, face_maps1 = d1,
                                deg_maps2 = s2, face_maps2 = d2)
    
def precompos0(biop):
    '''precomposes the operator with the bioperator s_0 x s_0'''
    
    return biop.compose(SimplicialBioperator(deg_maps1 = [0], 
                                             deg_maps2 = [0]))

def shi(n):
    '''returns all bioperators defining the chain homotopy between 
    EZAW and the identity'''
    
    if n == 0:
        return set()
    
    if n == 1:
        return {precompos0(prime(biop)) for biop in ezaw(n)}

    if n > 1:
        answer =  {precompos0(prime(biop)) for biop in ezaw(n)}
        answer ^= {prime(biop) for biop in shi(n-1)}
        
        return answer

### Example

For the map
\begin{equation*}
EZAW : C_\bullet(\Delta^2 \times \Delta^2) \to C_\bullet(\Delta^2 \times \Delta^2)
\end{equation*}

awe verify that

\begin{equation*}
EZAW([0,1,2] \otimes [0,1,2]) =  [0, 0, 0] \otimes [0, 1, 2] + [0, 0, 1] \otimes [1, 2, 2] + [0, 1, 1] \otimes [1, 1, 2] + [0, 1, 2] \otimes  [2, 2, 2].
\end{equation*}

In [38]:
print([biop([0,1,2], [0,1,2]) for biop in ezaw(2)])

[([0, 1, 1], [1, 1, 2]), ([0, 0, 0], [0, 1, 2]), ([0, 0, 1], [1, 2, 2]), ([0, 1, 2], [2, 2, 2])]


### Example

For the map
\begin{equation*}
SHI : C_\bullet(\Delta^2 \times \Delta^2) \to C_\bullet(\Delta^2 \times \Delta^2)
\end{equation*}

we verify that

\begin{equation*}
SHI([0,1] \times [0,1]) = [0, 0, 1] \times [0, 1, 1]
\end{equation*}
and

\begin{equation*}
SHI([0,1,2] \times [0,1,2]) = 
[0, 0, 0, 1] \times [0, 1, 2, 2] + 
[0, 0, 1, 1] \times [0, 1, 1, 2] + 
[0, 0, 1, 2] \times [0, 2, 2, 2] +
[0, 1, 1, 2] \times [0, 1, 2, 2]  
\end{equation*}

In [42]:
print([biop([0,1], [0,1]) for biop in shi(1) if not biop.is_degenerate])

print('\n and')

[biop([0,1,2], [0,1,2]) for biop in shi(2) if not biop.is_degenerate]

[([0, 0, 1], [0, 1, 1])]

 and


[([0, 0, 1, 2], [0, 2, 2, 2]),
 ([0, 0, 0, 1], [0, 1, 2, 2]),
 ([0, 1, 1, 2], [0, 1, 2, 2]),
 ([0, 0, 1, 1], [0, 1, 1, 2])]

## Verifying $AWEZ = id$ and $EZAW \sim id$ 

We define functions that verify that the composition

\begin{equation}
N_*(X) \otimes N_*(Y) 
\stackrel{EZ}{\longrightarrow} N_{*}(X \times Y)
\stackrel{AW}{\longrightarrow} N_*(X) \otimes N_*(Y)
\end{equation}

is the identity and that $SHI$ is a chain homotopy between the identity and the composition

\begin{equation}
N_*(X \times Y) 
\stackrel{AW}{\longrightarrow} N_*(X) \otimes N_*(Y)
\stackrel{EZ}{\longrightarrow} N_{*}(X \times Y).
\end{equation}

I.e.
\begin{equation}
\partial \, SHI + SHI \, \partial = id + EZAW.
\end{equation}

In [57]:
from collections import Counter

def is_awez_equal_to_id(p,q):
    '''returns True if the composition AWEZ restricted to bidegree 
    (p,q) is the identity and False otherwise'''

    comps = {biop1.compose(biop2) for biop1, biop2 in product(aw(p+q), ez(p,q))}
    
    non_deg_comps = {biop for biop in comps if  not biop.op1.is_degenerate 
                                            and not biop.op2.is_degenerate}
    
    if [non_deg_comps].sort() is [SimplicialBioperator()].sort():
        return True
    else:
        return False
    
def boundary(n):
    '''returns the bilinear operators defining the boundary in N_n(X x Y)'''
    
    return {SimplicialBioperator(face_maps1 = [i], 
                                 face_maps2 = [i]) for i in range(n+1)}

def is_ezaw_homotopic_to_id(n):
    '''returns True if in degree n the composition EZAW is chain homotopic to the 
    identity via SHI and False if not'''
    
    # \partial \circ SHI
    a = [biop1.compose(biop2) for biop1, biop2 in product(boundary(n+1), shi(n))]
    # SHI \circ \partial
    b = [biop1.compose(biop2) for biop1, biop2 in product(shi(n-1), boundary(n))]
    # EZAW
    c = list(ezaw(n))
    # id
    d = [SimplicialBioperator()]
    
    answer = Counter([str(biop) for biop in (a+b+c+d) if not biop.is_degenerate])
    
    # cheking all coefficients are 0 mod 2
    if {0} == {i%2 for i in answer.values()}:
        return True 
    else: 
        return False

In [58]:
p, q, n = 3,5,7 

print(f'is AWEZ equal to the identity in bidegree {p,q}?')

if is_awez_equal_to_id(p,q):
    print('Yes!')
    
else:
    print('No!')
    
print(f'is AWEZ chain homotopic to the identity in degree {n}?')

if is_ezaw_homotopic_to_id(n):
    print('Yes!')
    
else:
    print('No!')

is AWEZ equal to the identity in bidegree (3, 5)?
Yes!
is AWEZ chain homotopic to the identity in degree 7?
Yes!


## TO-DO

Add the Frobenius Structure as described [here](https://ncatlab.org/nlab/show/Frobenius+monoidal+functor)